# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 4

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task4`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [1]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ALTP4BRJD to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "afa7d636-7cc3-4954-a7a0-13024052145c",
    "id": "331a63a9-7d90-40d9-859a-ed50a5c9a2d2",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "d0ecd01b-d782-448e-bae0-c3cad0e0543a"
      },
      {
        "tenantId": "94c4857e-1130-4ab8-8eac-069b40c9db20"
      },
      {
        "tenantId": "f702a9dc-ae48-4dc7-8f0a-8155a6dfa4e5"
      }
    ],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "afa7d636-7cc3-4954-a7a0-13024052145c",
    "user": {
      "name": "yh08037@gmail.com",
      "type": "user"
    }
  }
]


## Step 1. Write the code

In [2]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

Preparing Q# environment...


In [3]:
teamname="iQuack"  # Update this field with your team name
task=["task4"]
slack_id="Dohun Kim"  # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [4]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task4_DumpMachineWrapper : qsharp.QSharpCallable = None
Task4_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 4 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [5]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

// Task 4. 
// (input will contain 7 qubits)
operation Task4(input : Qubit[], target : Qubit) : Unit is Adj {
    // let N = Length(input);
    // for i in 0 .. 3 .. 2^N - 1 {
    //     ControlledOnInt(i, X)(input, target);
    // }
    use aux = Qubit[12];

    within {
        CNOT(input[1], aux[0]);
        CNOT(input[3], aux[0]);
        CNOT(input[5], aux[0]);

        CCNOT(input[1], input[3], aux[1]);
        CCNOT(input[1], input[3], aux[1]);
        CCNOT(input[3], input[5], aux[1]);

        CNOT(input[0], aux[2]);
        CNOT(input[2], aux[2]);
        CNOT(input[4], aux[2]);
        CNOT(input[6], aux[2]);

        CCNOT(input[0], input[2], aux[3]);
        CCNOT(input[0], input[4], aux[3]);
        CCNOT(input[0], input[6], aux[3]);
        CCNOT(input[2], input[4], aux[3]);
        CCNOT(input[2], input[6], aux[3]);
        CCNOT(input[4], input[6], aux[3]);

        ControlledOnInt(15, X)(Subarray([0, 2, 4, 6], input), aux[4]);

        CNOT(aux[0], aux[5]);
        CNOT(aux[2], aux[5]);
        CNOT(aux[1], aux[6]);
        CNOT(aux[3], aux[6]);

        ControlledOnInt(0, X)(Subarray([5, 6], aux), aux[7]);
        
        CNOT(aux[0], aux[8]);
        CNOT(aux[1], aux[8]);
        CNOT(aux[2], aux[9]);
        CNOT(aux[3], aux[9]);

        ControlledOnInt(0, X)(Subarray([8, 9], aux), aux[10]);

        CNOT(aux[7], aux[11]);
        CNOT(aux[10], aux[11]);
        CCNOT(aux[7], aux[10], aux[11]);

    } apply {
        ControlledOnInt(1, X)(Subarray([4, 11], aux), target);
        ControlledOnInt(5, X)(Subarray([0, 1, 4], aux), target);
    }
}

In [6]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task4_DumpMachineWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task4(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task4_ResourceEstimationWrapper() : Unit {
    let N = 7;
    use (input, target) = (Qubit[N], Qubit());
    Task4(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [ ]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
qsharp.config["dump.truncateSmallAmplitudes"]=True
Task4_DumpMachineWrapper.simulate()

|00000000⟩	0.0883883476483185 + 0𝑖
|00000010⟩	0.0883883476483185 + 0𝑖
|00000100⟩	0.0883883476483185 + 0𝑖
|00000110⟩	0.0883883476483185 + 0𝑖
|00001000⟩	0.0883883476483185 + 0𝑖
|00001010⟩	0.0883883476483185 + 0𝑖
|00001100⟩	0.0883883476483185 + 0𝑖
|00001110⟩	0.0883883476483185 + 0𝑖
|00010000⟩	0.0883883476483185 + 0𝑖
|00010010⟩	0.0883883476483185 + 0𝑖
|00010100⟩	0.0883883476483185 + 0𝑖
|00010110⟩	0.0883883476483185 + 0𝑖
|00011000⟩	0.0883883476483185 + 0𝑖
|00011010⟩	0.0883883476483185 + 0𝑖
|00011100⟩	0.0883883476483185 + 0𝑖
|00011110⟩	0.0883883476483185 + 0𝑖
|00100000⟩	0.0883883476483185 + 0𝑖
|00100010⟩	0.0883883476483185 + 0𝑖
|00100100⟩	0.0883883476483185 + 0𝑖
|00100110⟩	0.0883883476483185 + 0𝑖
|00101000⟩	0.0883883476483185 + 0𝑖
|00101010⟩	0.0883883476483185 + 0𝑖
|00101100⟩	0.0883883476483185 + 0𝑖
|00101110⟩	0.0883883476483185 + 0𝑖
|00110000⟩	0.0883883476483185 + 0𝑖
|00110010⟩	0.0883883476483185 + 0𝑖
|00110100⟩	0.0883883476483185 + 0𝑖
|00110110⟩	0.0883883476483185 + 0𝑖
|00111000⟩	0.0883883476483185 + 0𝑖
|00111010⟩	0.0883883476483185 + 0𝑖
|00111100⟩	0.0883883476483185 + 0𝑖
|00111110⟩	0.0883883476483185 + 0𝑖
|01000000⟩	0.0883883476483185 + 0𝑖
|01000010⟩	0.0883883476483185 + 0𝑖
|01000100⟩	0.0883883476483185 + 0𝑖
|01000110⟩	0.0883883476483185 + 0𝑖
|01001000⟩	0.0883883476483185 + 0𝑖
|01001010⟩	0.0883883476483185 + 0𝑖
|01001100⟩	0.0883883476483185 + 0𝑖
|01001110⟩	0.0883883476483185 + 0𝑖
|01010000⟩	0.0883883476483185 + 0𝑖
|01010010⟩	0.0883883476483185 + 0𝑖
|01010100⟩	0.0883883476483185 + 0𝑖
|01010110⟩	0.0883883476483185 + 0𝑖
|01011000⟩	0.0883883476483185 + 0𝑖
|01011010⟩	0.0883883476483185 + 0𝑖
|01011100⟩	0.0883883476483185 + 0𝑖
|01011110⟩	0.0883883476483185 + 0𝑖
|01100000⟩	0.0883883476483185 + 0𝑖
|01100010⟩	0.0883883476483185 + 0𝑖
|01100100⟩	0.0883883476483185 + 0𝑖
|01100110⟩	0.0883883476483185 + 0𝑖
|01101000⟩	0.0883883476483185 + 0𝑖
|01101010⟩	0.0883883476483185 + 0𝑖
|01101100⟩	0.0883883476483185 + 0𝑖
|01101110⟩	0.0883883476483185 + 0𝑖
|01110000⟩	0.0883883476483185 + 0𝑖
|01110010⟩	0.0883883476483185 + 0𝑖
|01110100⟩	0.0883883476483185 + 0𝑖
|01110110⟩	0.0883883476483185 + 0𝑖
|01111000⟩	0.0883883476483185 + 0𝑖
|01111010⟩	0.0883883476483185 + 0𝑖
|01111100⟩	0.0883883476483185 + 0𝑖
|01111110⟩	0.0883883476483185 + 0𝑖
|10000000⟩	0.0883883476483185 + 0𝑖
|10000010⟩	0.0883883476483185 + 0𝑖
|10000100⟩	0.0883883476483185 + 0𝑖
|10000110⟩	0.0883883476483185 + 0𝑖
|10001000⟩	0.0883883476483185 + 0𝑖
|10001010⟩	0.0883883476483185 + 0𝑖
|10001100⟩	0.0883883476483185 + 0𝑖
|10001110⟩	0.0883883476483185 + 0𝑖
|10010000⟩	0.0883883476483185 + 0𝑖
|10010010⟩	0.0883883476483185 + 0𝑖
|10010100⟩	0.0883883476483185 + 0𝑖
|10010110⟩	0.0883883476483185 + 0𝑖
|10011000⟩	0.0883883476483185 + 0𝑖
|10011010⟩	0.0883883476483185 + 0𝑖
|10011100⟩	0.0883883476483185 + 0𝑖
|10011110⟩	0.0883883476483185 + 0𝑖
|10100000⟩	0.0883883476483185 + 0𝑖
|10100010⟩	0.0883883476483185 + 0𝑖
|10100100⟩	0.0883883476483185 + 0𝑖
|10100110⟩	0.0883883476483185 + 0𝑖
|10101000⟩	0.0883883476483185 + 0𝑖
|10101010⟩	0.0883883476483185 + 0𝑖
|10101100⟩	0.0883883476483185 + 0𝑖
|10101110⟩	0.0883883476483185 + 0𝑖
|10110000⟩	0.0883883476483185 + 0𝑖
|10110010⟩	0.0883883476483185 + 0𝑖
|10110100⟩	0.0883883476483185 + 0𝑖
|10110110⟩	0.0883883476483185 + 0𝑖
|10111000⟩	0.0883883476483185 + 0𝑖
|10111010⟩	0.0883883476483185 + 0𝑖
|10111100⟩	0.0883883476483185 + 0𝑖
|1011111100000000⟩	0.0883883476483185 + 0𝑖
|11000000⟩	0.0883883476483185 + 0𝑖
|11000010⟩	0.0883883476483185 + 0𝑖
|11000100⟩	0.0883883476483185 + 0𝑖
|11000110⟩	0.0883883476483185 + 0𝑖
|11001000⟩	0.0883883476483185 + 0𝑖
|11001010⟩	0.0883883476483185 + 0𝑖
|11001100⟩	0.0883883476483185 + 0𝑖
|11001110⟩	0.0883883476483185 + 0𝑖
|11010000⟩	0.0883883476483185 + 0𝑖
|11010010⟩	0.0883883476483185 + 0𝑖
|11010100⟩	0.0883883476483185 + 0𝑖
|11010110⟩	0.0883883476483185 + 0𝑖
|11011000⟩	0.0883883476483185 + 0𝑖
|11011010⟩	0.0883883476483185 + 0𝑖
|11011100⟩	0.0883883476483185 + 0𝑖
|11011110⟩	0.0883883476483185 + 0𝑖
|11100000⟩	0.0883883476483185 + 0𝑖
|11100010⟩	0.0883883476483185 + 0𝑖
|1

()

## Step 3. Evaluate the code using resource estimation

In [8]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
# qsharp.azure.connect(
#     resourceId="...",
#     location="...",
#     credential="CLI")
# If you're using this notebook in qBraid, keep it
targets = qsharp.azure.connect(
    resourceId="/subscriptions/331a63a9-7d90-40d9-859a-ed50a5c9a2d2/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/dohun-west",
    location="westus2")

Connecting to Azure Quantum...

KeyboardInterrupt: 

In [ ]:
qsharp.azure.target("microsoft.estimator")

In [ ]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task4_ResourceEstimationWrapper, jobName="RE for the task 4")

In [ ]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

In [ ]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [ ]:
evaluate_results(result)